In [1]:
top_k = 10


def decode(tokenizer, pred_idx, top_clean):
    ignore_tokens = string.punctuation + '[PAD]'
    tokens = []
    for w in pred_idx:
        token = ''.join(tokenizer.decode(w).split())
        if token not in ignore_tokens:
            tokens.append(token.replace('##', ''))
    return '\n'.join(tokens[:top_clean])


def encode(tokenizer, text_sentence, add_special_tokens=True):
    text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
    # if <mask> is the last token, append a "." so that models dont predict punctuation.
    if tokenizer.mask_token == text_sentence.split()[-1]:
        text_sentence += ' .'

    input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
    mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
    return input_ids, mask_idx

In [2]:
import torch
import string

from transformers import BertTokenizer, BertForMaskedLM
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
text_sentence="Here is a test about"
print(text_sentence)
text_sentence+= ' <mask>'
top_clean=5
input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
with torch.no_grad():
    predict = bert_model(input_ids)[0]
bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)
print()
print(bert)

Here is a test about

you
me
love
it
this


In [16]:
import torch
import string
from transformers import AutoTokenizer

from transformers import BertTokenizer, BertForMaskedLM
tokenizer_checkpoint = "./S288C_TOKENIZER"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint,local_files_only=True)
bert_model = BertForMaskedLM.from_pretrained('CUBERT').eval()

In [19]:
bert_model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [ ]:
from transformers import pipeline
import json

pipe = pipeline(task='fill-mask', model='./CUBERT',tokenizer="./S288C_TOKENIZER")
out = pipe(f"4 2 2 {pipe.tokenizer.mask_token} 2 1")
print(json.dumps(out, indent=4))

In [28]:
from transformers import pipeline

classifier = pipeline(task='fill-mask', model='./CUBERT',tokenizer="zluvolyote/CUBERT_TOKENIZER_Yeast")
classifier("1 [MASK] 2 3")

[{'score': 0.49071866273880005,
  'token': 5,
  'token_str': '1',
  'sequence': '1 1 2 3'},
 {'score': 0.2961772680282593,
  'token': 6,
  'token_str': '2',
  'sequence': '1 2 2 3'},
 {'score': 0.10418839007616043,
  'token': 7,
  'token_str': '3',
  'sequence': '1 3 2 3'},
 {'score': 0.06545155495405197,
  'token': 8,
  'token_str': '4',
  'sequence': '1 4 2 3'},
 {'score': 0.023255983367562294,
  'token': 9,
  'token_str': '5',
  'sequence': '1 5 2 3'}]